### Modeling Notebook - SVC

In [17]:
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
from sklearn.ensemble.forest import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import SelectFromModel, VarianceThreshold, chi2, f_classif
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

%matplotlib inline

In [2]:
df = pd.read_csv('cleaned_games_dataset.csv', index_col='Unnamed: 0')

In [3]:
df.head()

fg_home_1  fga_home_1  fg%_home_1  3p_home_1  3pa_home_1  3p%_home_1  \
1         31          78       0.397          7          23       0.304   
2         37          70       0.529         11          27       0.407   
5         31          72       0.431          6          21       0.286   
6         30          71       0.423          8          21       0.381   
9         35          79       0.443          7          16       0.438   

   ft_home_1  fta_home_1  ft%_home_1  orb_home_1       ...         \
1         28          38       0.737          18       ...          
2         18          27       0.667           8       ...          
5         27          33       0.818          13       ...          
6         34          42       0.810          16       ...          
9         13          15       0.867          11       ...          

   pts_away_3_opp  betting_line_away_3_opp  over_under_terms_3_opp  \
1             114                      9.5                  -107.0   
2              87                      8.0                  -105.0   
5              84                     -3.5                  -105.0   
6             102                     -3.5                  -104.0   
9              74                      8.0                  -105.0   

   over_under_3_opp  home_3_opp  game_total_score_3_opp  home_opp  \
1             195.0           0                     224         0   
2             189.5           1                     180         1   
5             188.0           1                     174         1   
6             195.0           0                     197         0   
9             188.0           1                     163         1   

   days_1_back_opp  days_2_back_opp  days_3_back_opp  
1                2                2                2  
2                3                1                2  
5                1                3                1  
6                1                3                1  
9                2                1                3  

[5 rows x 272 columns]

In [4]:
df.drop(['gid_today', 'game_total_score'], axis=1, inplace=True)

In [5]:
X = df[[x for x in df.columns if x != 'bet_won_over' and x != 'bet_won_under']]
y_over = df[['bet_won_over', 'game_date']]
y_under = df[['bet_won_under', 'game_date']]

In [6]:
X_train = X[X['game_date'] < '2017-09-01']
X_test = X[X['game_date'] > '2017-09-01']

y_train_over = y_over[y_over['game_date'] < '2017-09-01']
y_train_under = y_under[y_under['game_date'] < '2017-09-01']
y_test_over = y_over[y_over['game_date'] > '2017-09-01']
y_test_under = y_under[y_under['game_date'] > '2017-09-01']


In [7]:
X_train.drop('game_date', axis=1, inplace=True)
X_test.drop('game_date', axis=1, inplace=True)
y_train_over.drop('game_date', axis=1, inplace=True)
y_test_over.drop('game_date', axis=1, inplace=True)
y_train_under.drop('game_date', axis=1, inplace=True)
y_test_under.drop('game_date', axis=1, inplace=True)

/Users/jordanbailey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jordanbailey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/jordanbailey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can 

In [15]:
np.linspace(0,2,4)

array([0.        , 0.66666667, 1.33333333, 2.        ])

In [10]:
ss = StandardScaler()
svc = SVC()

In [12]:
pipe = Pipeline([
    ('ss', ss),
    ('svc', svc),
])

In [20]:
params = {
    'svc__C': np.linspace(0.1, 1, 5),
    'svc__kernel': ['linear', 'rbf']
}

In [21]:
gs = GridSearchCV(pipe, param_grid=params, scoring='roc_auc', cv=5)
gs.fit(X_train, y_train_over)

/Users/jordanbailey/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jordanbailey/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jordanbailey/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jordanbailey/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array w

/Users/jordanbailey/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jordanbailey/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jordanbailey/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jordanbailey/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array w

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('ss', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'svc__C': array([0.1  , 0.325, 0.55 , 0.775, 1.   ]), 'svc__kernel': ['linear', 'rbf']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [22]:
print(gs.best_score_)
print(gs.best_params_)

0.4896396184269182
{'svc__C': 0.1, 'svc__kernel': 'linear'}


In [23]:
gs.score(X_test, y_test_over)

0.5223435784694495